<a href="https://colab.research.google.com/github/hkyoo52/CV/blob/main/pytorch_module_%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Module

In [ ]:
#사측연산 : add, sub, dot, div
A=torch.Tensor([3])
B=torch.Tensor([10])
torch.div(A,B)

tensor([0.3000])

### gather
**기하적 생각**

* gather(행렬,dim,index) 식으로 사용

* dim은 몇번째 axis에서 한개씩 꺼낼 것인가?

* index는 그 axis에서 몇번째 index를 꺼낼 것인가?

  => 이때 shape는 dim 부분만 1 나머지는 output 행렬과 동일해야함

![image](https://user-images.githubusercontent.com/63588046/150920786-e6818bc9-f3f4-4a0b-b84e-9290cdd007c0.png)

=> index는 [[0,1,2],[0,1,2],[0,1,2]]이고 
    shape는 (3,1,3)으로 만들어야함 (output이 (3,3) 인 행렬이므로)


**산술적 생각**

* if dim==0   =>   out[i][j][k] = input[[index[i][j][k]][j][k]]   

* if dim==1   =>   out[i][j][k] = input[[i][index[i][j][k]]][k]]   

* if dim==2   =>   out[i][j][k] = input[[i][j][index[i][j][k]]]]  





In [ ]:
# 3d에서 대각행렬 값만 가져오는법
def get_diag_element_3D(A):
    c,h,w=A.size()
    print(c,w)
    index=torch.arange(min(h,w)).expand(c,min(h,w)).view(c,1,min(h,w))
    output = torch.gather(A,1,index).squeeze()
    return output

## 텐서

### 텐서 생성
torch.Tensor(직접 입력)

torch.from_numpy(numpy를 tensor로 바꿈)

torch.zeros(size) : size 크기의 tensor 생성

torch.zeros_like(행렬) : 행렬의 크기와 동일한 크기의 tensor 생성

In [ ]:
import torch
import numpy as np

# 텐서 생성하기
new_torch=torch.Tensor([[1,2],
                       [3,4]])
# numpy로 tensor 생성
a=np.array([[1,2],[3,4]])
new_torch=torch.from_numpy(a)

In [ ]:
# 행렬의 크기가 주어져있을때 그 크기에 맞춰서 생성
a=torch.zeros(2,3)
torch.zeros_like(a)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [ ]:
# 텐서 indexing 하는법
# torch.index_select(텐서, axis 방향, indexing(토치)할것)
indexing=torch.tensor([0])
torch.index_select(new_torch,1,indexing)

tensor([[1.],
        [3.]])

chunk(행렬,n,dim) : dim 방향으로 행렬을 n개로 쪼갬

swapdims(행렬,dim0,dim1) : dim0과 dim1을 transpose함


### Random sampling
https://pytorch.org/docs/stable/torch.html#random-sampling

torch.Tensor.bernoulli_(a), torch.Tensor.normal_(a)
 등등 존재


### Math operation
https://pytorch.org/docs/stable/torch.html#math-operations


In [ ]:
# einsum(equation,행렬) : 두개의 행렬을 곱해서 더하기 (크기는 equation으로)
a13=torch.Tensor([[1,2,3]])
b31=torch.Tensor([[2,2,2]])
torch.einsum('bi,bi -> i', a13, b31)  #1*3,1*3 -> 3

tensor([2., 4., 6.])

In [ ]:
a23=torch.Tensor([[1,2,3],[2,4,6]])
b13=torch.Tensor([[2,2,2]])
torch.einsum('bi,ci -> bi', a23, b13)   #2*3, 1*3 -> 2*3

tensor([[ 2.,  4.,  6.],
        [ 4.,  8., 12.]])

## 모듈 제작

``` python
class 함수명(nn.Module):
  def __init__(self,변수):
    super().__init__()
    self.변수 = 변수
    self.add_list=nn.ModuleList([함수,함수,함수]) 
  def forward(self,변수2):
    ~~~
    for i in self.add_list:
      x=i(x)
    return x

``` 
ModuleList만 변경해가면서 모델을 생성할 수 있다. (list대신 ModuleList를 사용해야 이후 list값이 안 사라지고 계속 사용 가능하다)


### 파라미터 생성

파라미터로 변수를 만들어야 값을 저장하고 미분을 할 수 있다!!

```
def __init__(self,~~):
  self.W=Parameter(torch.~~)
```

**Tensor**
* gradient X
* 값 업데이터 X
* 모델 값 저장 X

**Parameter**
* gradient O
* 값 업데이터 O
* 모델 값 저장 O

**Buffer**
* gradient X
* 값 업데이터 X
* 모델 값 저장 O

## 모듈 분석

model.named_modules() : 모듈 이름 생성

model.get_submodule('모듈 이름')

module.name_parameter()

module.get_parameter('파라미터이름')



```
def extra_repr(self):
  return '~={}'.format(~)
```
=> module에 파라미터 나오게 출력 가능

In [ ]:
import torch
from torch import nn
from torch.nn.parameter import Parameter

class Function_A(nn.Module):
    def __init__(self, name):
        super().__init__()
        self.name = name

    def forward(self, x):
        return x
    
    def extra_repr(self):
        return 'name={}'.format(self.name)

class Function_B(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x

class Layer_AB(nn.Module):
    def __init__(self):
        super().__init__()
        self.a = Function_A('duck')
        self.b = Function_B()

    def forward(self, x):
        x = self.a(x)
        x = self.b(x)
        return x

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.ab = Layer_AB()

    def forward(self, x):
        x = self.ab(x)
        return x

x=torch.tensor([3])
model=Model()

for name, module in model.named_modules():
  print('<Name> {}'.format(name))
  print('<Module> {}'.format(module))

<Name> 
<Module> Model(
  (ab): Layer_AB(
    (a): Function_A(name=duck)
    (b): Function_B()
  )
)
<Name> ab
<Module> Layer_AB(
  (a): Function_A(name=duck)
  (b): Function_B()
)
<Name> ab.a
<Module> Function_A(name=duck)
<Name> ab.b
<Module> Function_B()


In [ ]:
model.get_submodule('ab.a')

Function_A(name=duck)

## hook & apply
### hook
* 함수내에 파라미터에 순차적으로 적용해줌

### apply
* model.apply(함수)
* model 내에 모든 module(함수 등)에 각각 다 적용

In [ ]:
# hook 원리

def prog_A(x):
  return 1

def program_B(x):
    return x - 3

class Package(object):
  def __init__(self):
    self.program=prog_A
    self.hooks=[]

  def __call__(self,x):
    x=self.program(x)
    if self.hooks:
      for hook in self.hooks:
        output=hook(x)
        if output:
          x=output
    return x

package=Package()
input=3
output=package(input)
print('empty hook : ',package(input))
package.hooks.append(program_B)
print('append hook : ',package(input))


empty hook :  1
append hook :  -2


In [14]:
# register_forward_hook
# def hook(모듈,input,output) => return값은 output

import torch
from torch import nn
from torch.nn.parameter import Parameter

class Add(nn.Module):
    def __init__(self):
        super().__init__() 

    def forward(self, x1, x2):
        output = torch.add(x1, x2)

        return output

def hook(module,input,output):
  return output+5

add=Add()

add.register_forward_hook(hook)

x1=Parameter(torch.Tensor([1]),requires_grad=True)
x2=Parameter(torch.Tensor([2]),requires_grad=True)
print('결과값은 5가 더 더해짐 : x1+x2+5 = ',add(x1,x2))

결과값은 5가 더 더해짐 : x1+x2+5 =  tensor([8.], grad_fn=<AddBackward0>)


In [21]:
# 텐서 hook
# def tensor_hook(grad)
# 파라미터에 직접 출력



class Model(nn.Module):
    def __init__(self):
        super().__init__() 
        self.W=Parameter(torch.Tensor([5]))

    def forward(self, x1, x2):
        return self.W*(x1+x2)

answer=[]

def tensor_hook(grad):
  answer.append(grad)
  return 

model=Model()
model.W.register_hook(tensor_hook)

x1=Parameter(torch.Tensor([1]),requires_grad=True)
x2=Parameter(torch.Tensor([2]),requires_grad=True)

output=model(x1,x2)
output.backward()

print('model의 W의 기울기값 : ',answer)

model의 W의 기울기값 :  [tensor([3.])]


In [22]:
# module hook
# register_full_backward_hook
# return 값이 input

class Model(nn.Module):
    def __init__(self):
        super().__init__() 
        self.W=Parameter(torch.Tensor([5]))

    def forward(self, x1, x2):
        return self.W*(x1+x2)

def module_hook(module, grad_input, grad_output):
    grad_input=torch.divide(grad_input[0],sum(grad_input)),torch.divide(grad_input[1],sum(grad_input))
    return grad_input

model.register_full_backward_hook(module_hook)

x1=Parameter(torch.Tensor([1]),requires_grad=True)
x2=Parameter(torch.Tensor([2]),requires_grad=True)
output = model(x1, x2)
output.backward()

print('학습하는 도중에 hook을 사용했다.',x1.grad, x2.grad,x1.grad+x2.grad)

학습하는 도중에 hook을 사용했다. tensor([0.5000]) tensor([0.5000]) tensor([1.])
